## iris 붓꽃 데이터 세트분류

In [5]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [11]:
#data load
iris = load_iris()

iris_data = iris.data

iris_label = iris.target
print('iris target값:', iris_label)
print('iris target명:', iris.target_names)

#to dataframe
iris_df = pd.DataFrame(data=iris_data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df.head()

iris target값: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
iris target명: ['setosa' 'versicolor' 'virginica']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


X_train,X_test,y_train,y_test = train_test_split(iris_data,iris_label, test_size = 0.2 ,random_state = 11)

test_size는 전체 dataset 중 test set의 비율, random_state는 호출시 같은 세트를 호출하기 위한 난수

### 분류알고리즘인 의사결정 트리로 학습

In [22]:
#DecisionTreeClassifier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=11)
#학습 수행
dt_clf.fit(X_train,y_train)

DecisionTreeClassifier(random_state=11)

In [24]:
#학습이 완료된 DecisionTreeClassfier 객체에서 test data set로 예측 수행
pred = dt_clf.predict(X_test)

In [26]:
from sklearn.metrics import accuracy_score
print('예측 정확도 : {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도 : 0.9333


1. 데이터 세트 분리

2. 모델 학습

3. 예측 수행

4. 평가                ...순으로 진행했음

사이킷런은 분류 알고리즘 구현한 클래스를 Classifier, 회귀 알고리즘을 구현한 클래스를 Regressor로 지칭. 

이 두개를 Estimator 클래스라고 하며 내부에서 fit()과 predict() 구현

In [27]:
from sklearn.datasets import load_iris

iris_data = load_iris()
print(type(iris_data))

<class 'sklearn.utils.Bunch'>


In [28]:
keys = iris_data.keys()
print(keys)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [35]:
iris_data.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

## 교차 검증(Cross validation)

고정된 학습 데이터와 테스트 데이터로 평가를 하다보면 테스트 데이터에 편향되도록 모델을 유도할 수 있다.

교차검증은 이러한 데이터 편중을 막기 위해 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행하는 것이다. 

test set이 아닌 train set에서 다시 validation set을 빼서 학습시킨다. 여기서 K개의 데이터 폴드 세트를 만들어서 학습시키는 방법을 K폴드 교차 검증이라고 한다. ex) 5개의 set으로 나눈다고 하면 다섯개의 검증 set를 서로 다르게 구성한다음에 이들의 예측 값을 평균 낸 결과를 반영한다. 

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state = 156)

# 5개의 폴드 세트로 분리하는 KFOLD 객체와 폴드 세트별 정확도를 담을 리스트 생성
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 크기:', features.shape[0])

붓꽃 데이터 크기: 150


In [37]:
features.shape

(150, 4)

In [45]:
n_iter = 0
for train_index, test_index in kfold.split(features):
    #kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 데이터 추출
    X_train, X_test = features[train_index],features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    #반복시마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도:{1}, 학습 데이터 크기: {2}, 검증 데이터 크기:{3}'.format(n_iter,accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
#개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도:1.0, 학습 데이터 크기: 120, 검증 데이터 크기:30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도:0.9667, 학습 데이터 크기: 120, 검증 데이터 크기:30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도:0.8667, 학습 데이터 크기: 120, 검증 데이터 크기:30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도:0.9333, 학습 데이터 크기: 120, 검증 데이터 크기:30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도:0.7333, 학습 데이터 크기: 120, 검증 데이터 크기:30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


## Stratified K 폴드

만약 대출 사기 데이터 같은 분류 문제를 생각해보자. 부도문제와 마찬가지로 label 이 사기와 그렇지 않은 정상 상태인 0과 1 두 가지가 있다면 사기인 label 의 비율은 상대적으로 엄청 적을 것이다(Data imbalance가 있다). 그런데 train set에서 무작위로 k 개의 validation set을 나눠서 학습시킨다면 각각의 표본의 분포(label의 비율)이 달라질 것이고 학습에 영향을 미칠 것이다. 이 때 Stratified K폴드는 원본 데이터의 레이블 분포를 먼저 고려한 뒤 이 분포와 동일하게 train, validation set를 분배해준다.

In [48]:
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [49]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']): #k-fold와 다르게 label에 대한 정보까지 넣어줘야 얘가 비율조정함
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    16
Name: label, dtype: int64


그냥 k-fold로 했을때는 150개를 50개씩 나누는데 검증데이터에 label값이 각각 다르게 들어갔었는데 stratified k-fold사용하니 label의 비율을 맞춰준다.

사이킷런 api인 **cross_val_score()**사용하면 더 간편하게 이용가능

## GridSearchCV

하이퍼 파라미터 튜닝할때 사용. 여러 하이퍼 파라미터를 순차적으로 변경하면서 최적화한다. 

데이터 세트를 자동으로 coss-validation 을 위한 분리 수행함.

### 레이블 인코딩

문자열 값을 숫자형 카테고리 값으로 변환. 그러나 숫자 값에 크고 작음이 있는 경우는 사용하면 안된다. 숫자값의 위계가 있기때문에.

선형회귀와 같은 ML 알고리즘에는X,  트리 계열에는 OK

In [67]:
from sklearn.preprocessing import LabelEncoder

items = ['tv', '냉장고', '전자렌지' ,'컴퓨터', '선풍기', '선풍기' , '믹서', '믹서']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 변환값:', labels)

인코딩 변환값: [0 1 4 5 3 3 2 2]


In [66]:
print('인코딩클래스:', encoder.classes_)

인코딩클래스: ['tv' '냉장고' '믹서' '선풍기' '전자렌지']


### One-hot encoding

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np

items = ['tv', '냉장고', '전자렌지' ,'컴퓨터', '선풍기', '선풍기' , '믹서', '믹서']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
#2차원 데이터로 변환
labels = labels.reshape(-1,1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)
print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

원-핫 인코딩 데이터
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원-핫 인코딩 데이터 차원
(8, 6)


### 피처 스케일링과 정규화

In [17]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns = iris.feature_names)

print('feature 들의 평균 값')
print(iris_df.mean())
print('\nfeature 들의 분산 값')
print(iris_df.var())

feature 들의 평균 값
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

feature 들의 분산 값
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

#transform() 시 스케일 변환된 데이터 세트가 numpy ndarray로 반환되 이를 Dataframe으로 변환
iris_df_scaled = pd.DataFrame(data = iris_scaled, columns=iris.feature_names)

print('feature 들의 평균 값')
print(iris_df_scaled.mean())
print('\nfeature 들의 분산 값')
print(iris_df_scaled.var())

feature 들의 평균 값
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

feature 들의 분산 값
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


모든 칼럼들의 평균이 0에가까운값 , 분산이 1에 가까운값으로 변환되었음.

### MinMaxScaler

데이터를 0과 1사이의 값으로 변환시켜준다(음수값이 있으면 -1에서 1으로). 

데이터의 분포가 가우시안 분포가 아닐 경우에 Min, Max Scale을 적용해 볼 수 있다.

In [22]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
#MinMaxScaler로 데이터 세트 변환. fit()와 transform()호출
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

#transform() 시 스케일 변환된 데이터 세트가 numpy ndarray로 반환되 이를 Dataframe으로 변환
iris_df_scaled = pd.DataFrame(data = iris_scaled, columns = iris.feature_names)
print('feature 들의 최솟값 값')
print(iris_df_scaled.min())
print('\nfeature 들의 최댓값 값')
print(iris_df_scaled.max())

feature 들의 최솟값 값
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

feature 들의 최댓값 값
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


scaler 클래스의 fit(). transform()은 2차원 이상의 데이터만 가능하므로 reshape(-1,1)로 차원변경 실행했음.
*학습시킬 때 train set으로 fit과 transform 하는데 test set에 대해서 또다시 fit시키면 달라지기 때문에 기존 fit이 적용된 스케일링 기준 정보를 사용해야 한다.